In [1]:
import numpy as np

In [6]:
x = np.zeros([5,5])
x[1,0] = 1
x[2,0] = 1
x[4,0] = 1

x[0, 3] = 1
x[1, 3] = 1
x[2, 3] = 1
x

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [9]:
zero_mask = x[:, 0] == 1
three_mask = x[:, 3] == 1
print(zero_mask)
print(three_mask)
x[zero_mask & three_mask, 4] = 25
x

[False  True  True False  True]
[ True  True  True False False]


array([[ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  1., 25.],
       [ 1.,  0.,  0.,  1., 25.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.]])